In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
data = pd.read_csv('data/usds_atlas_cleaned.csv')








Which counties have the most **number** of food desert tracks (LowIncomeTract is 1)? Top 5

In [99]:
data.groupby('County')['LILATracts_1And10'].sum().sort_values(ascending=False).head(10).index.tolist()[:10]
#data.groupby('County')['LILATracts_1And10'].sum().sort_values(ascending=False).head(5).index.to_list()

['Harris',
 'Dallas',
 'Bexar',
 'Tarrant',
 'Hidalgo',
 'El Paso',
 'Cameron',
 'Travis',
 'Bell',
 'Jefferson']

Track the number of tracts that are food deserts with stricter definitions:
- LILATracts_halfAnd10, LILATracts_1And20, LILATracts_Vehicle. 

In [100]:
normal_def = data['LILATracts_1And10'].sum()
diff_def_data = data[(data['LILATracts_1And10'] == 0) & ((data['LILATracts_1And20'] == 1) | (data['LILATracts_Vehicle'] == 1) | (data['LILATracts_halfAnd10'] == 1))]
diff_def = diff_def_data.shape[0]
diff_def

print(f'{normal_def} tracts fall under the standard definition of a food desert')
print(f'{diff_def} tracts fall under alternative, slightly tweaked definitions of a food desert.')

1022 tracts fall under the standard definition of a food desert
1035 tracts fall under alternative, slightly tweaked definitions of a food desert.


What tracts are food deserts, but also have low vehicle access? Which ones are food deserts, but don't have low vehicle access? 

In [101]:
FD_and_LVA = data[(data['LILATracts_1And10'] == 1) & data['HUNVFlag'] == 1].shape[0]
FD_and_NLVA = data[(data['LILATracts_1And10'] == 1) & data['HUNVFlag'] == 0].shape[0]
NFD_and_LVA = data[(data['LILATracts_1And10'] == 0) & data['HUNVFlag'] == 1].shape[0]
NFD_and_NLVA = data[(data['LILATracts_1And10'] == 0) & data['HUNVFlag'] == 0].shape[0]


df_2x2 = pd.DataFrame(
    [[FD_and_LVA, FD_and_NLVA],
     [NFD_and_LVA, NFD_and_NLVA]],
    index=['FD (1And10=1)', 'Not FD (1And10=0)'],
    columns=['LVA (HUNV=1)', 'Non-LVA (HUNV=0)']
)
print(df_2x2)
# export
df_2x2
df_2x2.to_csv('fd_and_lva_grid.csv', index=False)

                   LVA (HUNV=1)  Non-LVA (HUNV=0)
FD (1And10=1)               415              4823
Not FD (1And10=0)           575              4663


What percent of all Texas tracts are food deserts?

In [102]:
percent = data[data['LILATracts_1And10'] == 1].shape[0]/data.shape[0] * 100

print(f'{round(percent,2)}% of tracts in Texas are food deserts')

19.51% of tracts in Texas are food deserts


How many total people live in food deserts in Texas?    

In [103]:
#This looks at first just the single, more specific definition 
FD_pop = data[data['LILATracts_1And10'] == 1]['Pop2010'].sum()

percent_FD = FD_pop/data['Pop2010'].sum()

print(f'Specific definition: {FD_pop} people live in food tracts.')
print(f'This is roughly {round(percent_FD*100,2)}% of the Texas population.')

#This includes all broader definitions of a food desert 
mask = (data['LILATracts_1And10'] == 1) | (data['LILATracts_1And20'] == 1) | \
       (data['LILATracts_Vehicle'] == 1) | (data['LILATracts_halfAnd10'] == 1)

inclusive_fd_pop = data[mask]['Pop2010'].sum()
total_pop = data['Pop2010'].sum()

print(f"Inclusive estimate: {inclusive_fd_pop/total_pop*100:.2f}% of the population")


Specific definition: 4926344 people live in food tracts.
This is roughly 19.59% of the Texas population.
Inclusive estimate: 38.28% of the population


rural v urban. How many food desert tracks are there in each? What percent of tracts in rural v urban texas are food deserts?  

In [104]:
urban_and_FD = data[(data['Urban'] == 1) & (data['LILATracts_1And10'] == 1)].shape[0]
rural_and_FD = data[(data['Urban'] == 0) & (data['LILATracts_1And10'] == 1)].shape[0]

percent_urban = urban_and_FD/data[data['Urban'] == 1].shape[0] * 100
percent_rural = rural_and_FD/data[data['Urban'] == 0].shape[0] * 100

print(f'{round(percent_urban,2)}% of urban tracts are food deserts')
print(f'{round(percent_rural,2)}% of rural tracts are food deserts')

22.44% of urban tracts are food deserts
9.02% of rural tracts are food deserts


In [105]:
####
# THIS IS SUPER IMPORTANT. IN ALL USES OF THE FdDes DATASET, KEEP IN MIND THAT
# THERE ARE NOW ONLY 170 COUNTIES. THIS MEANS THAT ABOUT 80 COUNTIES DON'T HAVE ANY 
# TRACTS THAT ARE FOOD DESERTS.

'''Which counties don't have any food desert tracts?'''

not_fd_df = data.groupby('County')['LILATracts_1And10'].sum()
not_fd_df = not_fd_df[not_fd_df == 0]
list_of_counties_with_no_fd_tracts = not_fd_df.index.to_list()
len(list_of_counties_with_no_fd_tracts)

84

Of all food deserts, what percent are urban vs. rural?

In [106]:
FdDes = data[data['LILATracts_1And10'] == 1]

urban_portion = FdDes[FdDes['Urban'] == 1].shape[0]/FdDes.shape[0] * 100
rural_portion = FdDes[FdDes['Urban'] == 0].shape[0]/FdDes.shape[0] * 100

print(f"""\
{urban_portion}% of food desert tracts are urban, while
{rural_portion}% are rural.""")

89.92172211350294% of food desert tracts are urban, while
10.078277886497064% are rural.


What counties have the most number of kids in low income tracts?

In [107]:
kids_in_Fd_df = FdDes.groupby('County')['TractKids'].sum().sort_values(ascending=False)

percent_kids_fd = FdDes.copy()
percent_kids_fd = percent_kids_fd.groupby('County')[['Pop2010', 'TractKids']].sum()
percent_kids_fd['Percent Kids'] = round((percent_kids_fd['TractKids']/percent_kids_fd['Pop2010'] * 100),2)
percent_kids_fd.sort_values(ascending=False, by='Percent Kids')

,Pop2010,TractKids,Percent Kids
County,,,
Webb,55568,21951.0,39.50
Coryell,26135,10224.0,39.12
Hidalgo,461260,166717.0,36.14
Maverick,34812,12209.0,35.07
Cameron,184305,64499.0,35.00
...,...,...,...
Sabine,5106,864.0,16.92
Brewster,3161,524.0,16.58
Hays,5233,824.0,15.75


Food desert vs. percent of minority population? (hispanic and black)

In [108]:
FDvsMin = data

cols = ['Pop2010','TractWhite', 'TractBlack', 'TractAsian', 'TractNHOPI', 'TractAIAN', 'TractOMultir', 'TractHispanic']
race_df = FDvsMin.groupby('County')[cols].sum()

for col in cols[1:]:
    race_df[f'Pct {col}'] = round((race_df[col]/race_df['Pop2010'] * 100),2)

new_df = race_df.loc[:, ['Pct TractWhite', 'Pct TractBlack', 'Pct TractAsian', 'Pct TractNHOPI', 'Pct TractAIAN', 'Pct TractOMultir', 'Pct TractHispanic']]

counties_highest_black_FD = new_df.sort_values(by=['Pct TractBlack'], ascending=False)
counties_highest_black_FD.index.tolist()[:10]

'''Harris',
 'Dallas',
 'Bexar',
 'Tarrant',
 'Hidalgo',
 'El Paso',
 'Cameron',
 'Travis',
 'Bell',
 'Jefferson'''

# Jefferson, Dallas, 

"Harris',\n 'Dallas',\n 'Bexar',\n 'Tarrant',\n 'Hidalgo',\n 'El Paso',\n 'Cameron',\n 'Travis',\n 'Bell',\n 'Jefferson"